## Part 2:

### Sub-genres
So far we have investigated how each rock artist is connected to each other through references on their wiki page. Each of these rock artists are associated with sub-genres based on the info box on their wiki page. Artists who shares a sub-genre likely have the same inspirations, or maybe they have colloborations. Therefore, a hypothesis can be made here, that artists that are characterized by the same sub-genre is more likely to be linked together compared to other artists.

### Testing the hypothesis with modularity

One way to test this hypothesis could be to use modularity. If we consider each collection of artists within a certain sub-genre as a community, then high modularity would indicate a strong community structure. This would suggest that artists are more densely connected within their sub-genres than between different sub-genres, supporting our hypothesis.
